In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
# Read the file 
df = pd.read_csv("files/spotify_dataset.csv")
df.head()

,Index,Highest Charting Position,Number of Times Charted,Week of Highest Charting,Song Name,Streams,Artist,Artist Followers,Song ID,Genre,...,Danceability,Energy,Loudness,Speechiness,Acousticness,Liveness,Tempo,Duration (ms),Valence,Chord
0,1,1,8,2021-07-23--2021-07-30,Beggin',"48,633,449",Måneskin,3377762,3Wrjm47oTz2sjIgck11l5e,"['indie rock italiano', 'italian pop']",...,0.714,0.8,-4.808,0.0504,0.127,0.359,134.002,211560,0.589,B
1,2,2,3,2021-07-23--2021-07-30,STAY (with Justin Bieber),"47,248,719",The Kid LAROI,2230022,5HCyWlXZPP0y6Gqq8TgA20,['australian hip hop'],...,0.591,0.764,-5.484,0.0483,0.0383,0.103,169.928,141806,0.478,C#/Db
2,3,1,11,2021-06-25--2021-07-02,good 4 u,"40,162,559",Olivia Rodrigo,6266514,4ZtFanR9U6ndgddUvNcjcG,['pop'],...,0.563,0.664,-5.044,0.154,0.335,0.0849,166.928,178147,0.688,A
3,4,3,5,2021-07-02--2021-07-09,Bad Habits,"37,799,456",Ed Sheeran,83293380,6PQ88X9TkUIAUIZJHW2upE,"['pop', 'uk pop']",...,0.808,0.897,-3.712,0.0348,0.0469,0.364,126.026,231041,0.591,B
4,5,5,1,2021-07-23--2021-07-30,INDUSTRY BABY (feat. Jack Harlow),"33,948,454",Lil Nas X,5473565,27NovPIUIRrOZoCHxABJwK,"['lgbtq+ hip hop', 'pop rap']",...,0.736,0.704,-7.409,0.0615,0.0203,0.0501,149.995,212000,0.894,D#/Eb


From the columns description in [kaggle](https://www.kaggle.com/sashankpillai/spotify-top-200-charts-20202021), some columns' data type is to change. Let's first change the columns name for easier use. 

In [3]:
# Define a function to rename a given column. Lowercase + underscored
def rename_column(col_name):
    list_col_name = col_name.lower().split(" ") # Turn the col name in a list of col names in lowercase
    rename_col = "_".join(list_col_name) # Join back the lowercase names with underscore
    return rename_col

In [4]:
# Rename all the columns of the dataframe
df_clean = df.copy() # Deep copy of the dataframe
renamed_cols = list(map(rename_column, list(df.columns))) # Create a list of the renamed columns
df_clean.columns = renamed_cols # Change the old column names with the new ones
df_clean.columns

Index(['index', 'highest_charting_position', 'number_of_times_charted',
       'week_of_highest_charting', 'song_name', 'streams', 'artist',
       'artist_followers', 'song_id', 'genre', 'release_date', 'weeks_charted',
       'popularity', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'liveness', 'tempo', 'duration_(ms)', 'valence',
       'chord'],
      dtype='object')

The week of highest charting is supposed to be a date. We will keep just the starting date, as we know that the end week will be after 7 days from the start.

In [5]:
# Split the two dates and keep the first one
df_clean.week_of_highest_charting = df_clean.week_of_highest_charting.apply(lambda x: x.split("--")[0])
# Change the data type from String to date
df_clean.week_of_highest_charting = df_clean.week_of_highest_charting.astype("datetime64")

In [6]:
# Check if the data type is correct, no output means it is correct
assert df_clean.week_of_highest_charting.dtype == np.dtype('datetime64[ns]')

The streams are to change in integer

In [23]:
# Change the streams column from String to integer
df_clean.streams = df_clean.streams.apply(lambda x: "".join(x.split(","))).astype("int64")

In [32]:
# Check the streams column is the correct data type, no output means is correct
assert df_clean.streams.dtype == "int64"

In [47]:
# Change artist followers from String to int. First we need to imput the empty values to 0. 
df_clean.artist_followers = df_clean.artist_followers.apply(lambda x: 0 if x == " " else x).astype("int64")

In [59]:
# Assert artist follower is the correct data type
assert df_clean.artist_followers.dtype == "int64"

Turn release date into datetime

In [80]:
# Function to imput the correct date values
def input_date(date_value):
    if date_value == " ": # If the value is empty we put nan
        date_value = np.nan
    elif len(date_value) == 4: # If the value is just the year we put month and day 
        date_value += "-01-01"
    elif len(date_value) == 7: # If the value is just the year and the month we put the day
        date_value += "-01"
    return date_value

In [81]:
# Convert the release date column into datetime
df_clean.release_date = df_clean.release_date.apply(input_date).astype("datetime64")

In [83]:
# Check if the data type is correct for release_date
assert df_clean.release_date.dtype == np.dtype("datetime64[ns]")

Turn Popularity into integer data type

In [94]:
# Imput empty values before changing the datatype
df_clean.popularity = df_clean.popularity.apply(lambda x : 0 if x == " " else x).astype("int64")

In [95]:
assert df_clean.popularity.dtypes == "int64"

In [126]:
# Select danceability, energy, loudness, speechiness, acousticness, liveness and tempo, turn them into float
cols = list(df_clean.iloc[:, 13:20].columns)
for col in cols:
    df_clean[col] = df_clean[col].apply(lambda x: np.nan if x == " " else x).astype("float64")

In [127]:
# Check the data type of the columns we just change
df_clean.iloc[:, 13:20].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1556 entries, 0 to 1555
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   danceability  1545 non-null   float64
 1   energy        1545 non-null   float64
 2   loudness      1545 non-null   float64
 3   speechiness   1545 non-null   float64
 4   acousticness  1545 non-null   float64
 5   liveness      1545 non-null   float64
 6   tempo         1545 non-null   float64
dtypes: float64(7)
memory usage: 85.2 KB


In [145]:
# Turn duration into integer
df_clean["duration_(ms)"] = df_clean["duration_(ms)"].apply(lambda x: 0 if x == " " else x).astype("int64")
# Imput the 0 values to be the average of the durations
duration_mean = int(np.mean(df_clean["duration_(ms)"])) # compute the mean, save it as an integerb
df_clean["duration_(ms)"] = df_clean["duration_(ms)"].apply(lambda x : duration_mean if x == 0 else x)

In [144]:
df_clean["duration_(ms)"]

0       211560
1       141806
2       178147
3       231041
4       212000
         ...  
1551    209320
1552    181930
1553    217307
1554    152784
1555    221307
Name: duration_(ms), Length: 1556, dtype: int64

In [149]:
# Change the valence column to the correct data type
df_clean.valence = df_clean.valence.apply(lambda x: np.nan if x == " " else x).astype("float64")

In [150]:
assert df_clean.valence.dtypes == "float64"

In [154]:
# Change the chord column to the correct data type
df_clean.chord.unique()

array(['B', 'C#/Db', 'A', 'D#/Eb', 'G#/Ab', 'F#/Gb', 'C', 'D', 'A#/Bb',
       'G', 'E', 'F', ' '], dtype=object)

In [158]:
# Replace the empty values with unknown
df_clean.chord.replace(" ", "UNKNOWN", inplace = True)
# Turn chord column into category datatype
df_clean.chord = df_clean.chord.astype